<a href="https://colab.research.google.com/github/un-silencing-slavery-at-rose-hall/project-website/blob/main/data/rose-hall-data-convert-from-excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from numpy import nan

In [97]:
file_url = "https://github.com/muziejus/rose-hall/blob/main/data/Comprehensive%20List%20of%20Enslaved%20People%20at%20Rose%20Hall%20Estate,%201817-1832.xlsx?raw=true"
# create a list of values prof. Naylor used as "N/A"
na_values = ["not_applicable", "not_specified", "--", " "]
# Read in Excel file.
df = pd.read_excel(
    file_url, 
    na_values=na_values)
# create a new list of column names that are easier to type
column_mapping = {
    'id_for_digital_project': "personId",
    'name': "name",
    'other_versions_of_name_in_the_records': "otherNames",
    'christian_names_often_combined_with_surname_in_1832_list_in_old_st_james_book': "christianNames",
    'names_of_married_women_and_surnames_for_children_listed_in_1832_list_in_old_st_james_book': "familyNames",
    'country': "country",
    'colour': "colour",
    'gender': "gender",
    'age_on_1817_list': "age1817List",
    'familial_connections': "familyNotes",
    'mother': "mother",
    'id_of_mother': "motherId",
    'grandmother': "grandmother",
    'id_of_grandmother': "grandmotherId",
    'great grandmother': "greatgrandmother",
    'id_of_great_grandmother': "greatgrandmotherId",
    'my_additional_comments': "comments",
    '_rose_hall_journal_info': "journalInfo",
    'age_1817_registry': "age1817",
    'age_1820_registry': "age1820",
    'age_1823_registry': "age1823",
    'age_1826_registry': "age1826",
    'age_1829_registry': "age1829", 
    'age_1832_registry': "age1832",
    'age_1832_list_in_old_st_james_book': "age1832List",
    'primary_duties_and_positions_1832_list_in_old_st_james_book': "duties",
    'condition_1832_list_in_old_st_james_book': "condition",
    'disposition_1832_list_in_old_st_james_book': "disposition",
    'valuation_1832_list_in_old_st_james_book': "valuation",
    'display_name': "displayName",
    'individual_profile': "profile"
    }

new_columns = [
    "birthYear",
    "arrivalYear",
    "exitYear",
    "ageAtExit",
    "calcAgeDiff",
    "arrivalReason",
    "exitReason",
    "dob",
    "dod"
    ]
columns = [c for c in column_mapping.keys() if column_mapping[c] != None]
# Rename columns.
df = df[columns].rename(columns=column_mapping)
# Drop supplemental rows that include information prof. Naylor added that will break the website.
df.drop([i for i in range(208,221)], inplace=True)

In [110]:
for column in ["age1820", "age1823", "age1826", "age1829", "age1832", "age1832List"]:
  df[column] = pd.Series(df[column], dtype="string")
  df[column] = df[column].fillna("")

In [111]:
for column in ["name", "country", "colour", "gender", "duties", "displayName", "profile"]:
    df[column] = df[column].str.strip()


In [113]:
def genderNan(x):
  if x is nan:
    return "Unknown"
  else:
    return x

df["gender"] = df["gender"].apply(genderNan)

In [114]:
def countryNan(x):
  if x == "Creole/African?":
    return "Inconsistent"
  elif x is nan:
    return "Unknown"
  else:
    return x

df["country"] = df["country"].apply(countryNan)

In [115]:
def colourNan(x):
  if x == "Negro/Sambo?" or x == "Sambo/Mulatto?":
    return "Inconsistent"
  elif x is nan:
    return "Unknown"
  else:
    return x

df["colour"] = df["colour"].apply(colourNan)

In [116]:
duties_dictionary = {'At Palmyra': "Artisans and Specialists",
 'Attending Small Stock': "Animal Caretakers",
 'Attending Young Children': "Domestic Workers",
 'Blacksmith': "Artisans and Specialists",
 'Carpenter': "Artisans and Specialists",
 'Cartman': "Artisans and Specialists",
 'Cartman and Field': "Artisans and Specialists",
 'Cattleboy': "Animal Caretakers",
 'Cook for gang': "Domestic Workers",
 'Cooper': "Artisans and Specialists",
 'Distiller': "Artisans and Specialists",
 'Distiller and Field': "Artisans and Specialists",
 'Domestic': "Domestic Workers",
 'Driveress': "Field Workers",
 'Field': "Field Workers",
 'Field and Grasscutter': "Field Workers",
 'Fisherman': "Artisans and Specialists",
 'G.H. Attendant': "Domestic Workers",
 'Grasscutter': "Field Workers",
 'Head Boiler': "Artisans and Specialists",
 'Head Carpenter': "Artisans and Specialists",
 'Head Cartman': "Artisans and Specialists",
 'Head Cooper': "Artisans and Specialists",
 'Head Driver': "Artisans and Specialists",
 'Head Penkeeper': "Animal Caretakers",
 'Hogmeat Gang': "Field Workers",
 'Hospital': "Artisans and Specialists",
 'Hospital Attendant and Midwife': "Artisans and Specialists",
 'Mason': "Artisans and Specialists",
 'Muleman': "Animal Caretakers",
 'Not at Work': "“Not at Work”",
 "Overseer's Cook": "Domestic Workers",
 "Overseer's House": "Domestic Workers",
 'Second Driver': "Artisans and Specialists",
 'Washerwoman': "Domestic Workers",
 'Watchman': "Artisans and Specialists",
 'With Mrs. Palmer': "Domestic Workers",
 nan: "Unknown"}

df['duty_category'] = df['duties'].apply(lambda x: duties_dictionary[x])

In [151]:
def numTest(value):
  return value.replace(".0", "").isnumeric()

# Parse birth year
def arrivalYearHunter(row):
  if row["age1817List"] > 0:
    return 1817
  elif numTest(row["age1820"]):
    return 1820 - float(row["age1820"])
  elif numTest(row["age1823"]):
    return 1823 - float(row["age1823"])
  elif numTest(row["age1826"]):
    return 1826 - float(row["age1826"])
  elif numTest(row["age1829"]):
    return 1829 - float(row["age1829"])
  elif numTest(row["age1832List"]):
    return 1832 - float(row["age1832List"])
  else:
    return 0

In [152]:
testdf = df
testdf["arrivalYear"] = testdf.apply(arrivalYearHunter, axis=1)

In [153]:
testdf["arrivalYear"].value_counts()

1817.0    152
0.0        27
1822.0      6
1818.0      5
1825.0      4
1828.0      4
1819.0      3
1831.0      3
1830.0      2
1821.0      2
Name: arrivalYear, dtype: int64

In [149]:
testdf["age1826"].value_counts()

                  178
6 mths              4
47-Dead             3
1.0                 3
38-Dead             2
4-Dead              2
10 days-Dead        1
20-Transported      1
46-Dead             1
50-Dead             1
58-Dead             1
77-Dead             1
57-Dead             1
2 1/2 years         1
77-dead             1
9-Dead              1
28-Dead             1
32-Dead             1
7 days-Dead         1
69-Dead             1
48-Dead             1
49-Dead             1
Name: age1826, dtype: Int64

In [8]:
import time
now = time.strftime("%Y-%m-%d-%H:%M")
df.to_csv(f"{now}-data.csv")

In [ ]:
json_string = df.to_json(orient="records")

output = "export default " + json_string + ";"
f = open("app/data/people.js", "w")
f.write(output)
f.close()

FileNotFoundError: ignored